CENP_B_ds_redWt Vs CENP_B_ds_red Q1012 

In [1]:
# import necessary modules
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import skimage.filters as filters
import skimage.color as color
from PIL import Image
import pandas as pd
from skimage.util import img_as_float
from skimage.color import label2rgb, rgb2gray,gray2rgb
from skimage import segmentation,morphology,feature,measure,filters
from scipy import ndimage as ndi
import glob
from natsort import natsorted 
import os
import glob
import re

In [2]:
# import function file
%run functions-Cytoplasm-Method2.ipynb

In [21]:
# path to the local folder for the notebooks
BASE_DIR="/Users/Trupti/01-LIDo/02-VijiProject/ImageAnalysis/" 
#path to the image folder

FOLDER="set4"
IMAGE_DIR=BASE_DIR + "data/AMovie/" + FOLDER + "/"
CSV_DIR = BASE_DIR + "AnalysisMethods/AnalysisResults/AMovie/" + FOLDER +"/" + "csvs/"



#### A List for the Images Folders

In [22]:
wt=[]
ctrl=[]
wt_path = IMAGE_DIR + 'wt'
ctrl_path=IMAGE_DIR + 'control'

for folder in glob.glob(wt_path + '/*'):
    wt.append(wt_path + '/' + folder.split('/')[-1].split('.')[0])
for folder in glob.glob(ctrl_path + '/*'):
    ctrl.append(ctrl_path + '/' + folder.split('/')[-1].split('.')[0])     


In [23]:
# For wt

for i in range(len(wt)):
    
    df_YFP_astrin,df_CENP_B_ds_red=method_cenpb(wt[i],FOLDER)#,YFP_astrin_cyto,CENP_B_ds_red_cyto,YFP_astrin_bg,CENP_B_ds_red_bg
    
    df=pd.merge(df_YFP_astrin, df_CENP_B_ds_red,  how='outer',on=['label','area']
                                                       ,suffixes=('_YFP_astrin', '_CENP_B_ds_red'))
    #df['YFP_astrin_cyto']=YFP_astrin_cyto
    #df['CENP_B_ds_red_cyto']=CENP_B_ds_red_cyto
    #df['YFP_astrin_bg']=YFP_astrin_bg
    #df['CENP_B_ds_red_bg']=CENP_B_ds_red_bg
    
    directory=wt[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs
    path = CSV_DIR + 'wt/' + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    #pd.concat([df1,df_cdki,df_dmso],ignore_index=True)
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from %s " %( directory))
   
    df.to_csv(path + '/' + 'particles'+'.csv') 
   

NaNs removed from Exp2018_032_IV_HeLa_MG132_YFP-AstrinWT_CENPB-dsRed_GFP_mcherry01_R3D_D3D 


In [24]:
# For control

for i in range(len(ctrl)):
    
    df_YFP_astrin,df_CENP_B_ds_red=method_cenpb(ctrl[i],FOLDER) #,YFP_astrin_cyto,CENP_B_ds_red_cyto,YFP_astrin_bg,CENP_B_ds_red_bg
    
    df=pd.merge(df_YFP_astrin, df_CENP_B_ds_red,  how='outer',on=['label','area']
                                                       ,suffixes=('_YFP_astrin', '_CENP_B_ds_red'))
    #df['YFP_astrin_cyto']=YFP_astrin_cyto
    #df['CENP_B_ds_red_cyto']=CENP_B_ds_red_cyto
    #df['YFP_astrinbg']=YFP_astrin_bg
    #df['CENP_B_ds_red_bg']=CENP_B_ds_red_bg
    
    directory=ctrl[i].split('/')[-1].split('.')[0] # to create a folder per experiment to save csvs
    path = CSV_DIR + 'ctrl/' + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    #pd.concat([df1,df_cdki,df_dmso],ignore_index=True)
    check_nan_in_df = df.isnull().values.any()
    if check_nan_in_df >0:
        df=df.dropna()
        print("NaNs removed from %s " %( directory))
    df.to_csv(path + '/' + 'particles'+'.csv') 
   

### Plots

In [25]:
import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("notebook",font_scale=1.5, rc={"lines.linewidth":2.5})

# path to save graphs
GRAPH_DIR=BASE_DIR + "AnalysisMethods/AnalysisResults/AMovie/" + FOLDER + "/" + "graphs/"




In [26]:
# plot graphs 
# dmso vs cdki
# The paths are for the generated csv files
wt_csv=[]
ctrl_csv=[]
wt_path = CSV_DIR + 'wt'
ctrl_path=CSV_DIR + 'ctrl'

for folder in natsorted(glob.glob(wt_path + '/*')):
    wt_csv.append(wt_path + '/' + folder.split('/')[-1].split('.')[0])
    
for folder in natsorted(glob.glob(ctrl_path + '/*')):
    ctrl_csv.append(ctrl_path + '/' + folder.split('/')[-1].split('.')[0])  
  

In [27]:
# read wildtype csvs to a dataframe
cell_count=0
df_graph_wt=pd.DataFrame()
for i in range(len(wt_csv)):
    df_wt=pd.read_csv(wt_csv[i]+'/'+'Particles.csv')
    df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto']= df_wt['mean_intensity_CENP_B_ds_red']/df_wt['CENP_B_ds_red_cyto']
    df_wt['YFP_astrin@kt/YFP_astrin@cyto']= df_wt['mean_intensity_YFP_astrin']/df_wt['YFP_astrin_cyto']
    df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto']= df_wt['CENP_B_ds_red@kt/CENP_B_ds_red@cyto'] / df_wt['YFP_astrin@kt/YFP_astrin@cyto']
    df_wt['condition']='WildType'
    df_wt['cell']=FOLDER+"_"+'wt_cell_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', wt_csv[i])
    df_wt['Experiment']= expt_name
    fname=FOLDER+'wt_'+ str(i)
    df_wt_final=outliers_astrin(df_wt)
    df_wt_final.to_csv(BASE_DIR  + 'GraphAnalysis/wt/' +FOLDER + '/' + fname +'.csv') 
    df_graph_wt=pd.concat([df_graph_wt,df_wt_final],ignore_index=True)    
    cell_count+=1
df_graph_wt = df_graph_wt.drop(columns=["Unnamed: 0"], axis=1)    

df_graph_wt.to_csv(BASE_DIR  + 'GraphAnalysis/wt/'+FOLDER+'/'+FOLDER+ "_"+'wt_astrin_graphs'+'.csv') 



In [28]:
cell_count=0
df_graph_ctrl=pd.DataFrame()
for i in range(len(ctrl_csv)):
    df_ctrl=pd.read_csv(ctrl_csv[i]+'/'+'Particles.csv')
    df_ctrl['CENP_B_ds_red@kt/CENP_B_ds_red@cyto']= df_ctrl['mean_intensity_CENP_B_ds_red']/df_ctrl['CENP_B_ds_red_cyto']
    df_ctrl['YFP_astrin@kt/YFP_astrin@cyto']= df_ctrl['mean_intensity_YFP_astrin']/df_ctrl['YFP_astrin_cyto']
    df_ctrl['CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto']= df_ctrl['CENP_B_ds_red@kt/CENP_B_ds_red@cyto'] / df_ctrl['YFP_astrin@kt/YFP_astrin@cyto']
    df_ctrl['condition']='Q1012'
    df_ctrl['cell']=FOLDER+"_"+'ctrl_cell_'+str(cell_count)
    expt_name=re.sub(r'^.+/([^/]+)$', r'\1', ctrl_csv[i])
    df_ctrl['Experiment']= expt_name
    fname=FOLDER+'ctrl_'+str(i)
    df_ctrl_final=outliers_astrin(df_ctrl)
    df_ctrl_final.to_csv(BASE_DIR  + 'GraphAnalysis/wt/' +FOLDER + '/' + fname +'.csv') 
    df_graph_ctrl=pd.concat([df_graph_ctrl,df_ctrl_final],ignore_index=True)  
    cell_count+=1
    
df_graph_ctrl = df_graph_ctrl.drop(columns=["Unnamed: 0"], axis=1)    

df_graph_ctrl.to_csv(BASE_DIR  + 'GraphAnalysis/wt/'+FOLDER+'/'+FOLDER+ "_" + 'ctrl_astrin_graphs'+'.csv') 

In [29]:
df_graph=pd.DataFrame()
df_graph=pd.concat([df_graph,df_graph_wt,df_graph_ctrl],ignore_index=True)
print(len(df_graph_wt))
print(len(df_graph_ctrl))
print(len(df_graph))
df_graph.head()
df_graph.to_csv(BASE_DIR  + 'GraphAnalysis/wt/'+FOLDER+'/' + FOLDER+ "_"+'astrin_wt_combined_graphs'+'.csv') 
#df_graph.head()

1583
1391
2974


In [15]:
# count the number of KTs in CDKi and DMSO
ctrl = df_graph[df_graph['condition']=='Q1012'].shape[0]
wt = df_graph[df_graph['condition']=='WildType'].shape[0]

### CENP_B_ds_red@kt/CENP_B_ds_red@cyto

In [ ]:
import seaborn as sns

title="CENP_B_ds_red@kt To CENP_B_ds_red@cyto - WildType Vs Q1012- Density"
plt.figure(figsize=(12,12))
sns.set_theme(style="whitegrid")

sns_plot=sns.kdeplot(data=df_graph, x="CENP_B_ds_red@kt/CENP_B_ds_red@cyto",hue='condition').set_title(title,fontsize=25)
sns_plot.figure.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)


title="CENP_B_ds_red@kt:CENP_B_ds_red@cyto - WildType Vs Q1012-Violin"
plt.figure(figsize=(12,12))
sns.set_theme(style="whitegrid")


# Draw a nested violinplot and split the violins for easier comparison
sns_plot=sns.violinplot(data=df_graph, x="condition", y="CENP_B_ds_red@kt/CENP_B_ds_red@cyto", hue="Experiment",
              inner="quart",
               ).set_title(title,fontsize=25)

sns_plot.figure.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)
sns.despine(left=True)





## YFP_astrin@kt:YFP_astrin@cyto

In [ ]:

title="YFP_astrin@kt:YFP_astrin@cyto -  WildType Vs Q1012- Density"
plt.figure(figsize=(12,12))
sns.set_theme(style="whitegrid")

sns_plot=sns.kdeplot(data=df_graph, x="YFP_astrin@kt/YFP_astrin@cyto",hue='condition').set_title(title,fontsize=25)
sns_plot.figure.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)

title="YFP_astrin@kt:YFP_astrin@cyto -  WildType Vs Q1012- Violin"
plt.figure(figsize=(12,12))
sns.set_theme(style="whitegrid")
# Draw a nested violinplot and split the violins for easier comparison
sns_plot=sns.violinplot(data=df_graph, x="condition", y="YFP_astrin@kt/YFP_astrin@cyto", hue="Experiment",
              inner="quart",
               ).set_title(title,fontsize=25)

sns_plot.figure.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)
sns.despine(left=True)

plt.show()

## CENP_B_ds_red@kt:CENP_B_ds_red@cyto To YFP_astrin@kt:YFP_astrin@cyto

In [ ]:
import seaborn as sns
title="CENP_B_ds_red@kt:CENP_B_ds_red@cyto To YFP_astrin@kt:YFP_astrin@cyto -WildType Vs Q1012- Density"
plt.figure(figsize=(12,12))
sns.set_theme(style="whitegrid")

sns_plot=sns.kdeplot(data=df_graph, x="CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto",hue='condition').set_title(title,fontsize=25)
sns_plot.figure.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)

title="CENP_B_ds_red@kt:CENP_B_ds_red@cyto To YFP_astrin@kt:YFP_astrin@cyto - WildType Vs Q1012- Violin"
plt.figure(figsize=(12,12))
sns.set_theme(style="whitegrid")
# Draw a nested violinplot and split the violins for easier comparison
sns_plot=sns.violinplot(data=df_graph, x="condition", y="CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto", hue="Experiment",
              inner="quart",
               ).set_title(title,fontsize=25)

sns_plot.figure.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)
sns.despine(left=True)

plt.show()

## YFP_astrin@kt/YFP_astrin@cyto:CENP_B_ds_red@kt/CENP_B_ds_red@cyto

In [ ]:
import seaborn as sns
title="YFP_astrin@kt:YFP_astrin@cyto To CENP_B_ds_red@kt:CENP_B_ds_red@cyto -WildType Vs Q1012- Density"
plt.figure(figsize=(12,12))
sns.set_theme(style="whitegrid")

sns_plot=sns.kdeplot(data=df_graph, x="YFP_astrin@kt/YFP_astrin@cyto:CENP_B_ds_red@kt/CENP_B_ds_red@cyto",hue='condition').set_title(title,fontsize=25)
sns_plot.figure.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)

title="YFP_astrin@kt:YFP_astrin@cyto To CENP_B_ds_red@kt:CENP_B_ds_red@cyto- WildType Vs Q1012- Violin"
plt.figure(figsize=(12,12))
sns.set_theme(style="whitegrid")
# Draw a nested violinplot and split the violins for easier comparison
sns_plot=sns.violinplot(data=df_graph, x="condition", y='YFP_astrin@kt/YFP_astrin@cyto:CENP_B_ds_red@kt/CENP_B_ds_red@cyto', hue="Experiment",
              inner="quart",
               ).set_title(title,fontsize=25)

sns_plot.figure.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)
sns.despine(left=True)

plt.show()

In [ ]:
plt.figure(figsize=(15,15))
sns.set(font_scale=1.5)
title='PairPlot - Condition'
sns_plot=sns.pairplot(df_graph, hue="condition")

replacements = {'mean_intensity_CENP_B_ds_red': 'CENP_B_ds_red_int', 
                'mean_intensity_YFP_astrin': 'YFP_astrin_int',
                'YFP_astrin@kt/YFP_astrin@cyto': 'YFP_astrin@kt/YFP_astrin@cyto',
                'CENP_B_ds_red@kt/CENP_B_ds_red@cyto': 'CENP_B_ds_red@kt/CENP_B_ds_red@cyto',
                'YFP_astrin@kt/YFP_astrin@cyto:CENP_B_ds_red@kt/CENP_B_ds_red@cyto':'YFP_astrin@kt/YFP_astrin@cyto:CENP_B_ds_red@kt/CENP_B_ds_red@cyto',
               'CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto':'CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto'}
for i in range(6):
    for j in range(6):
        xlabel = sns_plot.axes[i][j].get_xlabel()
        ylabel = sns_plot.axes[i][j].get_ylabel()
        if xlabel in replacements.keys():
            sns_plot.axes[i][j].set_xlabel(replacements[xlabel],rotation=90)
           
        if ylabel in replacements.keys():
            sns_plot.axes[i][j].set_ylabel(replacements[ylabel],rotation=0,ha="right")
            
            


sns_plot.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
title="PairPlot - Experiment"
sns.set(font_scale=1.5)

sns_plot=sns.pairplot(df_graph, hue="Experiment")

replacements = {'mean_intensity_CENP_B_ds_red': 'CENP_B_ds_red_int', 
                'mean_intensity_YFP_astrin': 'YFP_astrin_int',
                'YFP_astrin@kt/YFP_astrin@cyto': 'YFP_astrin@kt/YFP_astrin@cyto',
                'CENP_B_ds_red@kt/CENP_B_ds_red@cyto': 'CENP_B_ds_red@kt/CENP_B_ds_red@cyto',
                'YFP_astrin@kt/YFP_astrin@cyto:CENP_B_ds_red@kt/CENP_B_ds_red@cyto':'YFP_astrin@kt/YFP_astrin@cyto:CENP_B_ds_red@kt/CENP_B_ds_red@cyto',
               'CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto':'CENP_B_ds_red@kt/CENP_B_ds_red@cyto:YFP_astrin@kt/YFP_astrin@cyto'}
for i in range(6):
    for j in range(6):
        xlabel = sns_plot.axes[i][j].get_xlabel()
        ylabel = sns_plot.axes[i][j].get_ylabel()
        if xlabel in replacements.keys():
            sns_plot.axes[i][j].set_xlabel(replacements[xlabel],rotation=90)
        if ylabel in replacements.keys():
            sns_plot.axes[i][j].set_ylabel(replacements[ylabel],rotation=0,ha="right")

sns_plot.savefig(GRAPH_DIR+title, bbox_inches="tight",dpi=300)
plt.show()